In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

from tensorflow.keras import layers

import numpy as np
import time
import sys
import matplotlib.pyplot as plt

In [2]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [3]:
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, Aer, execute, IBMQ
from qiskit.tools.visualization import circuit_drawer
from qiskit.tools.visualization import plot_histogram
from qiskit.extensions.unitary import unitary
from qiskit.tools.monitor import job_monitor
from qiskit.compiler import transpile, assemble
from qiskit.providers.aer import QasmSimulator
S_simulator = Aer.backends(name = 'statevector_simulator')[0]
M_simulator = Aer.backends(name = 'qasm_simulator')[0]

backend = QasmSimulator(configuration = {'method' : 'density_matrix'})
M_simulator = backend


import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy import misc
import cv2

In [4]:
'''def quanvolution(param):
    q = QuantumRegister(4, name = 'q_r')
    a = QuantumRegister(1, name = 'a_r')
    c = ClassicalRegister(1, name = 'c_r')
    qc = QuantumCircuit(q, a, c, name = 'q_circ')

    qc.h(q)

    qc.u3(param[0, 0], param[0, 1], param[0, 2], q[0])
    qc.u3(param[1, 0], param[1, 1], param[1, 2], q[0])
    qc.u3(param[2, 0], param[2, 1], param[2, 2], q[0])
    qc.u3(param[3, 0], param[3, 1], param[3, 2], q[0])

    qc.mct(q, a, None, mode = 'noancilla')

    qc.measure(a[0], c)

    shots = 8192

    transpiled_circuit = transpile(qc, M_simulator, optimization_level = 1)
    job = M_simulator.run(assemble(transpiled_circuit, shots = shots))
    results = job.result()

    readout = results.get_counts()

    #print(readout.get('1', 0) / readout.get('0', shots))
    
    #qc.draw()
    
    return [readout.get('0', 0) / readout.get('1', shots)] * param.shape[-1]'''

In [4]:
def convolution(FOCUS, FILTER, shots = 8192):
    '''
    FOCUS = [[F00, F01],
             [F10, F11]]
    FILTER = [[FI00, FI01],
              [FI10, FI11]]
    '''
    q = QuantumRegister(4, name = 'q_r')
    a = QuantumRegister(1, name = 'a_r')
    c = ClassicalRegister(1, name = 'c_r')
    qc = QuantumCircuit(q, a, c, name = 'q_circ')

    qc.h(q)

    qc.u3(FOCUS[0, 0] * FILTER[0, 0], FOCUS[0, 0] * FILTER[0, 1], FOCUS[0, 0] * FILTER[1, 0], q[0])

    qc.u3(FOCUS[0, 1] * FILTER[0, 0], FOCUS[0, 1] * FILTER[0, 1], FOCUS[0, 1] * FILTER[1, 0], q[1])

    qc.u3(FOCUS[1, 0] * FILTER[0, 0], FOCUS[1, 0] * FILTER[0, 1], FOCUS[1, 0] * FILTER[1, 0], q[2])

    qc.u3(FOCUS[1, 1] * FILTER[0, 0], FOCUS[1, 1] * FILTER[0, 1], FOCUS[1, 1] * FILTER[1, 0], q[3])
    
    qc.h(q)

    qc.mct(q, a, None, mode = 'noancilla')

    qc.measure(a[0], c)

    #transpiled_circuit = transpile(qc, M_simulator, optimization_level = 1)
    #job = M_simulator.run(assemble(transpiled_circuit, shots = shots))
    job = execute(qc, M_simulator, shots = shots, optimization_level = 1)
    results = job.result()

    readout = results.get_counts()
    convolution = (readout.get('1', 0) / shots) * FILTER[1, 1]
    return convolution, readout, qc

In [13]:
def Qonv2D(filters = 1, kernel_size = (2, 2), stride = (1, 1), image = None):
    np.random.seed(465)
    N_FILTERS = filters
    KERNEL = kernel_size
    STRIDE = stride
    FILTERS = np.random.random(size = (N_FILTERS, KERNEL[0], KERNEL[1])) * np.pi
    CONV_SHAPE = ((image.shape[0] - KERNEL[0]) // STRIDE[0] + 1, (image.shape[0] - KERNEL[0]) // STRIDE[1] + 1, N_FILTERS)
    '''
    CONV_SHAPE = ((image.shape[0] - KERNEL[0]) // STRIDE[0] + 1, (image.shape[0] - KERNEL[0]) // STRIDE[1] + 1)
    CONV_OUTPUT = [] # shape = (Filters, CONV_IMAGE.shape)
    for FILTER in FILTERS:
        CONV_IMAGE = [] # shape = (((image.shape[0] - KERNEL[0]) // STRIDE[0] + 1, (image.shape[0] - KERNEL[0]) // STRIDE[1] + 1))
        for row in range(0, image.shape[0] - KERNEL[0] + 1, STRIDE[0]):
            for col in range(0, image.shape[1] - KERNEL[1] + 1, STRIDE[1]):
                focus = image[row : row + KERNEL[0], col : col + KERNEL[1]]
                convol = convolution(focus, FILTER, shots = 100)
                CONV_IMAGE.append(convol[0])
        CONV_OUTPUT.append(np.array(CONV_IMAGE).reshape(CONV_SHAPE))'''
    CONV_IMAGE = [[] for _ in range(N_FILTERS)] # shape = (((image.shape[0] - KERNEL[0]) // STRIDE + 1, (image.shape[0] - KERNEL[0]) // STRIDE + 1))
    for row in range(0, image.shape[0] - KERNEL[0] + 1, STRIDE[0]):
        for col in range(0, image.shape[1] - KERNEL[1] + 1, STRIDE[1]):
            for index, FILTER in enumerate(FILTERS):
                focus = image[row : row + KERNEL[0], col : col + KERNEL[1]]
                convol = convolution(focus, FILTER, shots = 100)
                CONV_IMAGE[index].append(convol[0])
    CONV_OUTPUT = np.stack(CONV_IMAGE, axis = -1)
    CONV_OUTPUT = CONV_OUTPUT.reshape(CONV_SHAPE)
    return CONV_OUTPUT

In [6]:
#quanvolution(np.random.normal(size = (4, 5)))

In [11]:
class Linear(layers.Layer):
    def __init__(self, units):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            name = "w_customized",
            shape = (input_shape[-1], self.units),
            initializer = "random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            name = "b_customized",
            shape = (self.units,), initializer="random_normal", trainable=True,
        )
        
    def get_config(self):
        config = super(Linear, self). get_config()
        return config

    def call(self, inputs):
        # Exotic calculations
        if tf.executing_eagerly():
            final_output = []
            for i in range(inputs.shape[0]):
                # Dummy operation
                #pred = quanvolution(np.random.normal(size = (4, inputs.shape[-1])))
                pred = [0] * 2
                final_output.append(list(pred))
                
            #tf.print(self.w, self.b)
            #return  tf.matmul(tf.convert_to_tensor(final_output, dtype = tf.float32), self.w) + self.b
            return  tf.matmul(inputs, self.w) + self.b
            #return tf.convert_to_tensor(final_output, dtype = "float32")
        return tf.matmul(inputs, self.w) + self.b
    
class MyReLu(layers.Layer):
    def __init__(self):
        super(MyReLu, self).__init__()
    def call(self, x):
        return tf.math.maximum(x, 0)

In [12]:
x = tf.ones((1, 4))
linear_layer = Linear(4)
y = linear_layer(x)
print(y)

tf.Tensor([[ 0.09874764 -0.10956707 -0.09820347  0.1926188 ]], shape=(1, 4), dtype=float32)


In [ ]:
def MyModel(width, height, depth, classes):
    input_shape = (height, width, depth)
    chanDim = -1
    
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding = "valid", input_shape = input_shape))
    model.add(Activation(tf.nn.relu))
    #model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "valid"))
    model.add(Activation(tf.nn.relu))
    #model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    '''model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = "same"))
    model.add(Activation(tf.nn.relu))
    model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2, 2)))'''
    
    model.add(Flatten())
    model.add(Dense(units = 10)) #### Custom Layer with gradient being recorded :D
    model.add(Activation(tf.nn.relu))
    #model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(units = classes))
    model.add(Activation(tf.nn.softmax))
    
    return model

In [ ]:
#@tf.function # Needed to make sure gradients are recorded
def step(X, y):
    # Keep track of our gradients
    with tf.GradientTape() as tape:
        # Make a prediction with the model and use it to calculate loss
        pred = model(X)
        loss = categorical_crossentropy(y, pred)
    # Calculate the gradient using our tape and then update the model weights
    grads = tape.gradient(loss, model.trainable_variables)
    #tf.print([v.name for v in model.trainable_variables])
    opt.apply_gradients(zip(grads, model.trainable_variables))

In [ ]:
# Initialize epochs, batch size and initial learning rate
EPOCHS = 2
BS = 128
INIT_LR = 0.001

# Loading MNIST
((x_train, y_train), (x_test, y_test)) = mnist.load_data()

# Adding a channel dimension and scaling
x_train = np.expand_dims(x_train, axis = -1)
x_test = np.expand_dims(x_test, axis = -1)
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# One-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
# Building model and initialize optimizer
model = MyModel(28, 28, 1, 10)
opt = Adam(learning_rate = INIT_LR, decay = INIT_LR / EPOCHS)

In [ ]:
model.summary()

In [ ]:
# Compute number of bacthes updates per epoch
numUpdates = x_train.shape[0] // BS

# Looping over the number of epochs
for epoch in range(EPOCHS):
    print(f"[INFO] starting epoch {epoch + 1}/{EPOCHS}...", end = "")
    sys.stdout.flush()
    epochStart = time.time()
    
    # Looping over the data in batch size increments
    for i in range(numUpdates):
        # Determine starting and ending slice indexes for the current batch
        start = i * BS
        end = start + BS
        
        # Take a step
        step(x_train[start: end], y_train[start: end])
    
    # Show timing information for the epoch
    epochEnd = time.time()
    elapsed = (epochEnd - epochStart) / 60.0
    print(f"took {elapsed:.4} minutes")

In [ ]:
# In order to calculate accuracy using Keras' functions we first need to compile the model
model.compile(optimizer = opt, loss = categorical_crossentropy, metrics = ["acc"])

# Now that the model is compiled we can compute the accuracy
(loss, acc) = model.evaluate(x_test, y_test)
print(f"[INFO] test accuracy: {acc:.4}")

In [ ]:
model.fit(x_train, y_train, batch_size = BS, epochs = EPOCHS * 1)

In [ ]:
history = model.history

In [ ]:
plt.plot(history.history['acc'])
#plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()